# RAG Pipeline with Incremental Indexing
This notebook demonstrates how to combine indexing and RAG pipelines to process URLs one at a time.

In [1]:
# Required libraries
!pip install haystack-ai
!pip install "datasets>=3.6.0"
!pip install "sentence-transformers>=4.1.0"
!pip install accelerate


In [2]:
from haystack import Pipeline
from haystack.utils.auth import Secret
from haystack.components.converters import HTMLToDocument
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.components.embedders.cohere import CohereDocumentEmbedder, CohereTextEmbedder
from haystack.components.evaluators import ContextRelevanceEvaluator,FaithfulnessEvaluator
from haystack.components.evaluators import SASEvaluator
from haystack.components.builders import PromptBuilder, AnswerBuilder
from haystack_integrations.components.rankers.cohere import CohereRanker


import os
from dotenv import load_dotenv
load_dotenv()

import json
import warnings
import pandas as pd
from urllib.parse import urlparse

from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv()

# 从.env文件读取API key
cohere_api_key = os.getenv("COHERE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["COHERE_API_KEY"] = cohere_api_key if cohere_api_key else ""
os.environ["OPENAI_API_KEY"] = openai_api_key if openai_api_key else ""

In [3]:
import json

# 读取本地 index_table.json 文件
with open("../files/index_table.json", "r", encoding="utf-8") as f:
    data = json.load(f)

## Prompt Engineering

In [4]:
prompt = """
You are a privacy policy expert. You are provided with {{app_url}}, which contains the privacy policy document for an app.
Your task is to:
 - answer the question based on the privacy policy document,
 - provide references for your answers based on the section in the privacy policy document from which your answer is generated,
 - produce your results strictly in the JSON format below (no extra text beyond JSON),
 - ensure that the 'url' in the 'meta' section is exactly {{app_url}}.

JSON format:
{
   "meta": {
       "id": {{ app_id }},
       "url": {{ app_url }},
       "title": {{ app_name | tojson }}
   },
   "reply": {
       "qid": "{{ qid }}",
       "question": "{{ question | escape }}",
       "answer": {
           "full_answer": "{{ full_answer | escape }}",
           "simple_answer": "{{ simple_answer | escape }}",
           "extended_simple_answer": {
               "comment": "{{ extended_comment | escape }}",
               "content": "{{ extended_content | escape }}"
           }
       },
       "analysis": "{{ analysis | escape }}",
       "reference": "{{ reference | escape }}"
   }
}


Instructions:
1. Approach each question systematically:
   a. Understand the question: Break down the question into specific components or sub-questions if needed.

   b. Identify relevant context from the privacy policy.
   c. Analyze the context and link it back to the question.
   d. Formulate the answer for each JSON field.
   e. Provide references:  (original text + 'URL: {{app_url}}'). If none, report 'N/A. URL: {{app_url}}'. Note that the original text must come from only the relevant context in page {{app_url}}

2. Output structure:
   - full_answer: must integrate info from both simple_answer and extended_simple_answer. The full answer section must not be empty.
   - simple_answer: follow the rules above. This section must not be empty.
   - extended_simple_answer: follow the rules above, or empty if not specified
   - analysis: describe your reasoning
   - reference: original text snippets + URL. The context must come from the app URL {{app_url}}. Attach the {{app_url}} in the end. Ensure JSON compatibility by replacing double quotes with single quotes.


3. Special rules for the `simple_answer` and `extended_simple_answer` fields:
   - If the question is “1. Does the app declare the collection of data?”:
       * simple_answer: "Yes" or "No"
       * extended_simple_answer: leave empty

   - If the question is “2. If the app declares the collection of data, what type of data does it collect?':
       * simple_answer: "NOTUSED"
       * extended_simple_answer: 
           - comment: "data collected"
           - content: list of data types collected
       * full_answer: MUST be a concise natural language synthesis of the listed data types (DO NOT use "NOTUSED" here; never leave it empty).

   - If the question is "3. Does the app declare the purpose of data collection and use?":
       * simple_answer: "Yes" or "No"
       * extended_simple_answer: leave empty

   - If the question is "4. Can the user opt out of data collection or delete data?":
       * simple_answer: "Yes" or "No"
       * extended_simple_answer: leave empty

   - If the question is "5. Does the app share data with third parties?":
       * simple_answer: "Yes" or "No"
       * extended_simple_answer: leave empty

   - If the question is "6. If the app shares data with third parties, what third parties does the app share data with?": 
       * simple_answer: "NOTUSED"
       * extended_simple_answer:
           - comment: "third parties"
           - content: list of third parties
       * full_answer: MUST be a concise natural language synthesis of the listed data types (DO NOT use "NOTUSED" here; never leave it empty).

Context:
{% for doc in documents %}
  {{ doc.content }}
  URL: {{ doc.meta['url'] }}
{% endfor %}

Question: {{ query }}
Answer:
"""

In [5]:
output_folder = 'outputs/'
os.makedirs(output_folder, exist_ok=True)

In [6]:
# Initialize document store and pipelines
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
import nltk
nltk.download('punkt')


# ===================== 静态无状态组件（循环外建一次） =====================
# 为防止部分站点拒绝默认UA，添加浏览器UA，并在失败时不抛出异常
fetcher = LinkContentFetcher(raise_on_failure=False)
converter = HTMLToDocument()
cleaner = DocumentCleaner()

splitter = DocumentSplitter(
    split_by="word",
    split_length=220,
    split_overlap=50
)

embedder = CohereDocumentEmbedder(
    model="embed-english-v3.0",
    api_base_url=os.getenv("CO_API_URL")
)
query_embedder = CohereTextEmbedder(
    model="embed-english-v3.0",
    api_base_url=os.getenv("CO_API_URL")
)

prompt_builder = PromptBuilder(
    template=prompt,
    required_variables=["query", "app_id", "app_url", "question", "documents"]
)
generator = OpenAIGenerator(model="gpt-3.5-turbo")
answer_builder = AnswerBuilder()


reranker = CohereRanker(model="rerank-english-v3.0", top_k=5)


# ===================== 占位的可变组件（先绑定一个临时 store） =====================
# 关键：writer / retriever 的实例是“可变”的，我们每轮替换它们的 document_store 即可
_initial_store = InMemoryDocumentStore()
writer = DocumentWriter(document_store=_initial_store, policy=DuplicatePolicy.OVERWRITE)
retriever = InMemoryEmbeddingRetriever(document_store=_initial_store, top_k=15)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xdn13\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# ===================== Pipeline 只构建一次（无 None） =====================
# 索引管道：fetch -> convert -> clean -> split -> embed -> write
indexing = Pipeline()
indexing.add_component("fetcher", fetcher)
indexing.add_component("converter", converter)
indexing.add_component("cleaner", cleaner)
indexing.add_component("splitter", splitter)
indexing.add_component("embedder", embedder)
indexing.add_component("writer", writer)  # 注意：是真实 writer，不是 None

indexing.connect("fetcher.streams", "converter.sources")
indexing.connect("converter", "cleaner")
indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "embedder")
indexing.connect("embedder", "writer")

# RAG 管道：query_embedder -> retriever -> prompt -> generator -> answer_builder
rag = Pipeline()
rag.add_component("query_embedder", query_embedder)
rag.add_component("retriever", retriever)  # 注意：是真实 retriever，不是 None
rag.add_component("reranker", reranker)    # ✅ 新增重排器
rag.add_component("prompt", prompt_builder)
rag.add_component("generator", generator)
rag.add_component("answer_builder", answer_builder)

rag.connect("query_embedder.embedding", "retriever.query_embedding")
# ✅ 先召回 → 再重排 → 再送给 prompt / answer_builder
rag.connect("retriever.documents", "reranker.documents")
rag.connect("reranker.documents", "prompt.documents")
rag.connect("reranker.documents", "answer_builder.documents")
rag.connect("prompt", "generator")
rag.connect("generator.replies", "answer_builder.replies")
# （不要连 query_embedder.text -> answer_builder.query）


🚅 Components
  - query_embedder: CohereTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reranker: CohereRanker
  - prompt: PromptBuilder
  - generator: OpenAIGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reranker.documents (list[Document])
  - reranker.documents -> prompt.documents (List[Document])
  - reranker.documents -> answer_builder.documents (List[Document])
  - prompt.prompt -> generator.prompt (str)
  - generator.replies -> answer_builder.replies (list[str])

## Use LLM generate replies

In [ ]:
# ===================== Retrieval short query mapping (Step B added) =====================
# Note: These are "short phrases for vector retrieval", removing template words and keeping only semantic core, currently not used
SHORT_QUERY_PROMPTS = {
    # ... (mapping skipped for brevity, currently not used)
}

# Define output folder
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Allow user to customize selection quantity  
num_to_process = 1  # Modify this to set how many URLs to process
if num_to_process:
    data = data[:num_to_process]
# Document excerpt length setting
excerpt_max_len = 150

In [ ]:
# Fallback for answer_json construction, immediately add this "secondary repair"
def repair_answer_json(answer_json):
    fa = answer_json.get("full_answer", "")
    # Only try if full_answer is string and looks like JSON
    if isinstance(fa, str) and fa.strip().startswith("{"):
        s = fa.strip()
        # 1) Remove common "trailing commas"
        s = re.sub(r",\s*}", "}", s)
        # 2) Remove fences ```json ... ```
        s = re.sub(r"```json\s*", "", s)
        s = re.sub(r"```\s*$", "", s).strip()
        try:
            cand = json.loads(s)
            # If cand itself is our expected structure (contains reply/answer), use it directly
            if "reply" in cand and "answer" in cand["reply"]:
                # Preserve existing qid to avoid losing question number
                if "qid" in answer_json and "qid" not in cand["reply"]:
                    cand["reply"]["qid"] = answer_json["qid"]
                return cand
        except:
            pass
    return answer_json

# Generate candidate fetch URLs (http/https, trailing slash, common privacy paths)
def expand_url_variants(url_raw: str) -> List[str]:
    variants = []
    u = url_raw.strip()
    if not u:
        return variants
    variants.append(u)  # Original
    # Trailing slash variant
    if u.endswith("/"):
        variants.append(u.rstrip("/"))
    else:
        variants.append(u + "/")
    # http variant
    if u.startswith("https://"):
        variants.append(u.replace("https://", "http://", 1))
    elif u.startswith("http://"):
        variants.append(u.replace("http://", "https://", 1))
    # Common privacy paths (if given site homepage or wrong path)
    from urllib.parse import urlparse
    parsed = urlparse(u)
    base = f"{parsed.scheme}://{parsed.netloc}"
    for path_suffix in ["/privacy", "/privacy-policy", "/privacypolicy"]:
        variants.append(base + path_suffix)
    return list(dict.fromkeys(variants))

In [10]:
# ===================== 主循环：每个 App 独立 store，但 pipeline 不重建 =====================
for idx, item in enumerate(data[:num_to_process]):
    app_id = item.get('id')
    app_name = item.get('title')
    app_url = item.get('url')
    if not app_url:
        print(f"No URL found for App ID: {app_id}, skipping...")
        continue

    print(f"Processing Document {idx + 1}/{num_to_process}: {app_name} ({app_id})")

    # —— 核心：为当前 App 创建独立的 store，并“就地替换” writer/retriever 所引用的 store —— #
    current_store = InMemoryDocumentStore()
    writer.document_store = current_store          # 索引写入到当前 store
    retriever.document_store = current_store       # 检索从当前 store 读取


    # 索引当前 app 的页面（含多URL回退，避免 400/403/404 直接失败）
    success = False
    for try_url in _generate_candidate_urls(app_url):
        try:
            _ = indexing.run({"fetcher": {"urls": [try_url]}})
            # 若写入成功应有文档
            if getattr(current_store, "count_documents", None):
                if current_store.count_documents() > 0:
                    success = True
                    if try_url != app_url:
                        print(f"[indexing] Fallback URL used: {try_url}")
                    break
            else:
                # 不支持计数则以不报错视为成功
                success = True
                break
        except Exception as e:
            print(f"[indexing] Failed on {try_url}: {e}")
            continue

    if not success:
        print(f"⚠️ Unable to fetch any content for App ID {app_id}. Skipping this app.")
        continue

    answers_list = []

    for j in tqdm(range(len(questions)), desc=f"|Processing Questions for {app_name}", unit="question"):
        query = f"""
        You are analyzing the '{app_name}' with URL: {app_url}.
        Answer the following questions based on the privacy policy document:
        {questions[j]}
        """
        # Step B: 为检索构造“短查询” —— 更聚焦、更稳健
        retriever_query = query_map.get(j, questions[j])

        result = rag.run({
            # ✅ 用短查询进行向量召回
            "query_embedder": {"text": questions[j]},
            # ✅ 放大召回池；重排器会压到 top_k=5（见上面的 reranker.top_k）
            "retriever": {"top_k": 15},
            "reranker": {"query": questions[j]},  # ✅ 为 CohereRanker 提供必需的 query
            "prompt": {"qid": f"q{j+1}", "query": query, "app_id": app_id, "app_url": app_url, "question": questions[j]},
            "answer_builder": {"query": query}
        })

        generated_answers = result['answer_builder']['answers']

        source_docs_export = []
        retrieved_context_list = []

        if generated_answers:
            structured_answer = generated_answers[0]
            answer = structured_answer.data
            source_documents = structured_answer.documents

            print(f"Question {j+1} answered using {len(source_documents)} document (forced single)")
            print(f"Query: {structured_answer.query}")

            for d in source_documents:
                excerpt = (d.content or "")[:500]
                if len(d.content or "") > 500: excerpt += "..."
                source_docs_export.append({
                    "id": getattr(d, 'id', None),
                    "score": getattr(d, 'score', None),
                    "excerpt": excerpt,
                    "url": (getattr(d, 'meta', {}) or {}).get('url')
                })
                retrieved_context_list.append(excerpt)
        else:
            # 回退到直接使用生成器输出
            answer = result['generator']['replies'][0]
            retrieved_context_list.append("No context available")

        # 解析与存储答案
        try:
            answer_json = json.loads(answer)
        except json.JSONDecodeError:
            # 兜底：保留题号与原文，避免评估阶段“未找到问题 qX 的RAG输出”
            answer_json = {
                "meta": {"id": app_id, "url": app_url},
                "reply": {
                    "qid": f"q{j+1}",
                    "question": questions[j],
                    "answer": {
                        "full_answer": str(answer),
                        "simple_answer": "",
                        "extended_simple_answer": {"comment": "", "content": ""}
                    },
                    "_parsing_note": "model_output_not_valid_json_fallback_used"
                }
            }
        
        # 在你设置完兜底 answer_json 之后调用：
        answer_json = _try_salvage_nested_json(answer_json)

        # 安全合并来源片段（如有）
        try:
            if source_docs_export:
                answer_json.setdefault("source_documents", source_docs_export)
        except Exception:
            pass

        answers_list.append(answer_json)



    # 保存 JSON
    output_file_path = os.path.join(output_folder, f"{app_id}.json")
    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(answers_list, f, ensure_ascii=False, indent=4)

    print(f"\nSaved answers for App ID {app_id} to {output_file_path}")

Processing Document 1/1: None (1361356590)


Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]
|Processing Questions for None:  17%|█▋        | 1/6 [00:04<00:21,  4.27s/question]

Question 1 answered using 5 document (forced single)
Query: 
        You are analyzing the 'None' with URL: http://www.balanceapp.com/balance-privacy.html.
        Answer the following questions based on the privacy policy document:
        1. Does the app declare the collection of data?
        


|Processing Questions for None:  33%|███▎      | 2/6 [00:07<00:14,  3.62s/question]

Question 2 answered using 5 document (forced single)
Query: 
        You are analyzing the 'None' with URL: http://www.balanceapp.com/balance-privacy.html.
        Answer the following questions based on the privacy policy document:
        2. If the app declares the collection of data, what type of data does it collect?
        


...

Question 4 answered using 5 document (forced single)
Query: 
        You are analyzing the 'None' with URL: http://www.balanceapp.com/balance-privacy.html.
        Answer the following questions based on the privacy policy document:
        4. Can the user opt out of data collection or delete data?
        


|Processing Questions for None:  83%|████████▎ | 5/6 [00:18<00:03,  3.78s/question]

Question 5 answered using 5 document (forced single)
Query: 
        You are analyzing the 'None' with URL: http://www.balanceapp.com/balance-privacy.html.
        Answer the following questions based on the privacy policy document:
        5. Does the app share data with third parties?
        


|Processing Questions for None: 100%|██████████| 6/6 [00:21<00:00,  3.58s/question]

Question 6 answered using 5 document (forced single)
Query: 
        You are analyzing the 'None' with URL: http://www.balanceapp.com/balance-privacy.html.
        Answer the following questions based on the privacy policy document:
        6. If the app shares data with third parties, what third parties does the app share data with?
        

Saved answers for App ID 1361356590 to outputs/1361356590.json


In [ ]:
# 🎯 Specialized RAG Evaluation Pipeline for Privacy Policy Questions
print("🎯 Creating specialized RAG evaluation pipeline for privacy policy questions...")

import os
import json
import pandas as pd
from haystack import Pipeline
from haystack.components.evaluators.document_mrr import DocumentMRREvaluator
from haystack.components.evaluators.faithfulness import FaithfulnessEvaluator
from haystack.components.evaluators.sas_evaluator import SASEvaluator

# Define questions and their corresponding ground truth format
PRIVACY_POLICY_QUESTIONS = {
    'q1': {
        'text': "1. Does the app declare the collection of data?",
        'type': 'binary',
        'expected_simple_answers': {'y': 'Yes', 'n': 'No'}
    },
    'q2': {
        'text': "2. What type of data does it collect?",
        'type': 'open'  # Open-ended question, groundtruth is free text
    },
    'q3': {
        'text': "3. Does the app declare the purpose of data collection and use?",
        'type': 'binary',
        'expected_simple_answers': {'y': 'Yes', 'n': 'No'}
    },
    'q4': {
        'text': "4. Can you opt out of data collection or delete data?",
        'type': 'binary',
        'expected_simple_answers': {'y': 'Yes', 'n': 'No'}
    },
    'q5': {
        'text': "5. Does the app share data with third parties?",
        'type': 'binary',
        'expected_simple_answers': {'y': 'Yes', 'n': 'No'}
    },
    'q6': {
        'text': "6. If the app shares data with third parties, what third parties does the app share data with?",
        'type': 'open'  # Open-ended question, groundtruth is free text
    }
}

def load_groundtruth_data(groundtruth_path='../groundtruth.json'):
    """Load groundtruth data"""
    with open(groundtruth_path, 'r', encoding='utf-8') as f:
        gt_data = json.load(f)
    
    # Convert to more usable format
    gt_dict = {}
    for item in gt_data:
        app_id = item['id']
        gt_dict[app_id] = {
            'q1': item['q1'],
            'q2': item['q2'], 
            'q3': item['q3'],
            'q4': item['q4'],
            'q5': item['q5'],
            'q6': item['q6']
        }
    
    print(f"✅ Loaded groundtruth annotations for {len(gt_dict)} applications")
    return gt_dict

def load_rag_output(app_id, outputs_dir='outputs'):
    """Load RAG output for a single application"""
    output_file = os.path.join(outputs_dir, f"{app_id}.json")
    
    if not os.path.exists(output_file):
        print(f"⚠️ Output file does not exist: {output_file}")
        return None
    
    try:
        with open(output_file, 'r', encoding='utf-8') as f:
            rag_data = json.load(f)
        return rag_data
    except Exception as e:
        print(f"❌ Failed to load RAG output {output_file}: {e}")
        return None

def extract_question_mapping(rag_outputs):
    """Extract question mapping from RAG output (mapping by question number prefix 1.~6.)"""
    question_mapping = {}
    for i, output in enumerate(rag_outputs):
        try:
            qtext = output['reply']['question'].strip()
            if qtext.startswith("1."):
                question_mapping['q1'] = output
            elif qtext.startswith("2."):
                question_mapping['q2'] = output
            elif qtext.startswith("3."):
                question_mapping['q3'] = output
            elif qtext.startswith("4."):
                question_mapping['q4'] = output
            elif qtext.startswith("5."):
                question_mapping['q5'] = output
            elif qtext.startswith("6."):
                question_mapping['q6'] = output
        except Exception as e:
            print(f"⚠️ Error processing output #{i+1}: {e}")
            continue
    return question_mapping

def create_privacy_evaluation_pipeline():
    """Create evaluation pipeline containing only Faithfulness / SAS / Context Relevance"""
    eval_pipeline = Pipeline()
    eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator())
    eval_pipeline.add_component("sas_evaluator", SASEvaluator(model="sentence-transformers/all-MiniLM-L6-v2"))
    eval_pipeline.add_component("context_relevance", ContextRelevanceEvaluator())
    return eval_pipeline

def evaluate_single_app(app_id, gt_dict, outputs_dir='outputs'):
    """Evaluate a single application"""
    print(f"\n📋 Evaluating Application ID: {app_id}")
    
    # Load data
    gt_answers = gt_dict.get(app_id)
    if not gt_answers:
        print(f"❌ Groundtruth for application {app_id} not found")
        return None
    
    rag_outputs = load_rag_output(app_id, outputs_dir)
    if not rag_outputs:
        print(f"❌ RAG output for application {app_id} not found")  
        return None

PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


🎯 创建基于5个隐私政策问题的专门RAG评估Pipeline...
🎯 开始执行隐私政策RAG评估...
🚀 开始隐私政策RAG系统综合评估
✅ 加载了 10 个应用的groundtruth标注
📊 找到 9 个可评估的应用
🎯 将评估前 9 个应用: [1361356590, 1435692352, 1458846512, 1493155192, 1498229813, 1588978095, 1665348316, 6447095050, 6474216442]

📋 评估应用 ID: 1361356590
✅ 找到 6 个问题的输出
🎯 运行Haystack评估pipeline...


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
100%|██████████| 6/6 [00:10<00:00,  1.76s/it]
PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


✅ Haystack评估完成

📋 评估应用 ID: 1435692352
✅ 找到 6 个问题的输出
🎯 运行Haystack评估pipeline...


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
100%|██████████| 6/6 [00:08<00:00,  1.45s/it]
PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


...

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
100%|██████████| 6/6 [00:09<00:00,  1.57s/it]
PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


✅ Haystack评估完成

📋 评估应用 ID: 1665348316
✅ 找到 6 个问题的输出
🎯 运行Haystack评估pipeline...


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
100%|██████████| 6/6 [00:10<00:00,  1.67s/it]
PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


✅ Haystack评估完成

📋 评估应用 ID: 6447095050
✅ 找到 6 个问题的输出
🎯 运行Haystack评估pipeline...


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
 50%|█████     | 3/6 [00:06<00:06,  2.10s/it]